<a href="https://colab.research.google.com/github/MohibShaikh/Action-Detector-using-LSTM/blob/main/Action_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *1. Installing Dependencies*

In [ ]:
!pip install tensorflow
!pip install opencv-python
!pip install mediapipe
!pip install scikit-learn
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 34.9 MB/s eta 0:00:00:00:0100:01


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time
import mediapipe as mp

2024-04-14 21:21:16.806377: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 21:21:16.806547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 21:21:16.955591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### *2. Keypoints using MP Holistic*

In [ ]:
mp_holistic = mp.solutions.holistic       # Holistic Model
mp_drawing = mp.solutions.drawing_utils   # Drawing Utilities

In [ ]:
def draw_landmarks(image, results):
    '''Draws face, pose, left hand, and right hand connections.'''
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS)

In [ ]:
def styled_landmarks(image, results):
    '''Draws face, pose, left hand, and right hand connections.'''
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp.holistic.FACE_CONNECTIONS
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1,
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp.holistic.POSE_CONNECTIONS
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness=1,circle_radius=4,
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness=1,circle_radius=2)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp.holistic.HAND_CONNECTIONS
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness=1,circle_radius=4,
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness=1,circle_radius=2)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS
                             mp_drawing.DrawingSpec(color=(245,192,30),thickness=1,circle_radius=4,
                             mp_drawing.DrawingSpec(color=(234,90,200),thickness=1,circle_radius=2)

In [ ]:
def mediapipe_detection(image, model):
    '''Takes an image and a mediapipe model, processes it, makes a prediction, returns a result image, and a prediction'''
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   #  Converting to BGR on which the model prediction happens
    image.flag.writeable = False                     # Image not writeable anymore (doing this saves memory)
    results = model.process(image)                   # Make Prediction
    image.flag.writeable = True                      # Image now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)   # Converting back to originall color
    return image, results

In [ ]:
cap = cv2.VideoCapture(2)
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened:

        # Reads the image
        ret,frame = cap.read()

        # Makes a predition
        image, results = mediapipe_detection(frame, holistic)

        # Draw Landmarks
        styled_landmarks(image, results)

        # Displays the output image
        cv2.imshow('OpenCV Feed',image)

        if cv2.waitkey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

### Key Value Points Extraction

In [ ]:
results.pose_landmarks

In [ ]:
poses=[]
for res in results.pose_landmarks.landmark:
    test = np.array(res.x,res.y,res.z,res.visibility)
    poses.append(test)

In [ ]:
def extract_keypoints(results):
    poses = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if face_landmarks else np.zeroes(33*4)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if left_hand_landmarks else np.zeroes(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if right_hand_landmarks else np.zeroes(21*3)
    face =np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if face_landmarks else np.zeroes(1404)
    return concatenate([poses,lh,rh,face])

### *4. Setup Folders for Collection*

In [ ]:
DATA_PATH = os.path.join('MP_DATA')

actions = ['hello','iloveyou','thanks']

no_squences = 30

sequence_length = 30